In [1]:
import os, sys, glob, logging
import inspect, urllib

import pandas, scipy, numpy

import time, datetime, pprint


In [2]:
logging.basicConfig(level=logging.INFO)
# logging.info('logging info')
# logging.warning('logging.warning')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# __file__ does not work in a jupyter notebook
# make something equivalent - actually only for this particular cell

import inspect
print(inspect.getfile.__doc__)
print()
src_fn = inspect.getfile(lambda: None)
print(os.path.abspath(src_fn))

Work out which source or compiled file an object was defined in.

/mnt/c/git_work/Xpress/<ipython-input-2-e8f0a355878d>


In [15]:

logging.info("logging")

# __file__ does not work in a jupyter notebook
# make something equivalent - actually only for this particular cell
__file__ = os.path.abspath(inspect.getfile(lambda: None))

# GTEx data from https://gtexportal.org/home/datasets
# Gene transcripts per million data
download_dir = '/home/zeno/c_drive/Users/Zeno/Downloads'
cache_dir = os.path.join(os.path.dirname(__file__), 'data_cache')
if not os.path.exists(cache_dir):
    logging.info('Createing {}'.format(cache_dir))
    os.makedirs(cache_dir, mode=0o777)

# Detailed data file:
# This file is quite large ~ 1GB
gtex_tpm_link = 'https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz'
gtex_tpm_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz')
if not os.path.exists(gtex_tpm_fn):
    print("downloading {}".format(gtex_tpm_link))
    #urllib.request.urlretrieve(gtex_tpm_link, gtex_tpm_fn)
# Just looking at median values by tissue (SMTSD) greatly reduces file size
gtex_tpm_med_link = 'https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct.gz'
gtex_tpm_med_fn = os.path.join(cache_dir, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct.gz')
if not os.path.exists(gtex_tpm_med_fn):
    print("downloading {}".format(gtex_tpm_med_link))
    urllib.request.urlretrieve(gtex_tpm_med_link, gtex_tpm_med_fn)
gtpm_med = pandas.read_csv(gtex_tpm_med_fn, sep='\t', skiprows=2, low_memory=False)

# Sample annotation data
# Main sample data of interest is:
# 'SMTS': 'Tissue Type, area from which the tissue sample was taken.  This is a parent value to SMTSD.'
# 'SMTSD': 'SMTS Detailed'
gtex_attr_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SampleAttributesDS.txt'
gtex_attr_fn = os.path.join(cache_dir, 'GTEx_v7_Annotations_SampleAttributesDS.txt')
gtex_attr_desc_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx'
gtex_attr_desc_fn = os.path.join(cache_dir, 'GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx')
if not os.path.exists(gtex_attr_fn):
    print('downloading {}'.format(gtex_attr_link))
    urllib.request.urlretrieve(gtex_attr_link, gtex_attr_fn)
if not os.path.exists(gtex_attr_desc_fn):
    print('downloading {}'.format(gtex_attr_desc_link))
    urllib.request.urlretrieve(gtex_attr_desc_link, gtex_attr_desc_fn)

    
gs = pandas.read_csv(gtex_attr_fn, sep='\t')

tissue_types = sorted(gs['SMTS'].unique())

tissue_dict = dict()
for tissue in tissue_types:
    tissue_dict[tissue] = sorted(gs[gs['SMTS'] == tissue]['SMTSD'].unique())

tissue_subtypes = sorted(gs['SMTSD'].unique())

print(tissue_types)

# Subject Phenotype data
# Age, sex and Hardy Scale death circumstances
gtex_pheno_link = 'https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SubjectPhenotypesDS.txt'
gtex_pheno_fn = os.path.join(cache_dir, 'GTEx_v7_Annotations_SubjectPhenotypesDS.txt')


# Just getting a few rows of this giant file
#gtpm_head = pandas.read_table(gtex_tpm_fn, skiprows=2, sep='\t', nrows=1000, usecols=range(3))
GTPM_SKIP = 2
gtpm_ = pandas.read_table(gtex_tpm_fn, skiprows=2, sep='\t', usecols=range(4))

# Other annotation data

# Mutually exclusive 
# Which study was this from?
mut_info_fn = os.path.join(cache_dir, 'enrichments-analysis-result.txt')
#mut_info = pandas.read_csv(mut_info_fn, sep='\t')


# Genes of interest
TEST_GENES = [
    'FAM138A',  # highly expressed
    'TP53',
    'ERBB2',  # Herceptin target
    'EGFR',
    'AKT1',
    'IL6',
    'APOE',
]

INFO:root:logging


['Adipose Tissue', 'Adrenal Gland', 'Bladder', 'Blood', 'Blood Vessel', 'Bone Marrow', 'Brain', 'Breast', 'Cervix Uteri', 'Colon', 'Esophagus', 'Fallopian Tube', 'Heart', 'Kidney', 'Liver', 'Lung', 'Muscle', 'Nerve', 'Ovary', 'Pancreas', 'Pituitary', 'Prostate', 'Salivary Gland', 'Skin', 'Small Intestine', 'Spleen', 'Stomach', 'Testis', 'Thyroid', 'Uterus', 'Vagina']


In [16]:
def gtpm_row_by_genename(gene_name):
    sub_table = gtpm_[gtpm_['Description'] == gene_name]
    return [GTPM_SKIP + 1 + i for i in sub_table.index][0]

def gtpm_skip_rows(row_ndx, gene_list=TEST_GENES):
    if row_ndx == GTPM_SKIP:
        return False  # header
    if row_ndx in range(GTPM_SKIP):
        return True
    for gn in gene_list:
        if row_ndx in gtpm_row_by_genename(gn):
            return False
    return True

def gtpm_keep_rows(gene_list=TEST_GENES):
    print(gene_list)
    return [GTPM_SKIP] + [gtpm_row_by_genename(gene) for gene in gene_list]

keep_rows = gtpm_keep_rows()
print(keep_rows)

['FAM138A', 'TP53', 'ERBB2', 'EGFR', 'AKT1', 'IL6', 'APOE']
[2, 6, 43806, 44842, 21315, 39030, 20796, 49176]


In [17]:
print(keep_rows)

start_time = time.time()
gtpm = pandas.read_table(gtex_tpm_fn, skiprows=lambda x: x not in keep_rows, sep='\t', header=0, usecols=range(100))

#gtpm = pandas.read_table(gtex_tpm_fn, skiprows=gtpm_skip_rows, sep='\t',)
end_time = time.time()

print("Tmp table took {} minutes".format((end_time - start_time)/60))


[2, 6, 43806, 44842, 21315, 39030, 20796, 49176]
Tmp table took 0.5383188088734945 minutes


In [22]:
# Find some representative gene sets.

# First find a set of benchmork genes.  Gene with consistent expression that can be used for scaling.

In [23]:
# Do something with the median values
# pick a single tissue per tissue type
sample_columns = [tissue_dict[t][0] for t in tissue_dict.keys()
                  if tissue_dict[t][0] != 'Cells - Leukemia cell line (CML)']
#print(gtpm_med.head())
gm = gtpm_med.drop('gene_id', axis=1).set_index('Description')
rename_cols = dict([(tissue_dict[t][0], t) for t in tissue_dict.keys()])
gm = gm[sample_columns].rename(columns=rename_cols)
# Copy table so we can add and modify values
gm = gm.copy()
gm.head()
# Using gm table we can chacterize the genes a bit.
# the spread includes extreme values, most of the tissues are similar
# Mean is ~ 16 - 17 TPM, but 50th percentile is 0 and 75th percentile is ~ 2
pandas.set_option('max_columns', 60)
gm.describe()

,Adipose Tissue,Adrenal Gland,Bladder,Blood,Blood Vessel,Brain,Breast,Cervix Uteri,Colon,Esophagus,Fallopian Tube,Heart,Kidney,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Salivary Gland,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina
count,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000,56202.000000
mean,16.701955,17.107780,16.663300,16.778048,16.730996,17.055343,16.138433,15.397327,17.005512,16.871584,16.455277,16.846769,15.812127,15.895570,16.124924,16.570206,16.956319,16.813498,16.520740,15.984667,16.227906,14.989199,16.591146,15.809636,16.516674,15.245102,17.319775,16.608929,16.584184,15.826178
std,375.574426,582.642470,351.055527,252.584699,221.139552,793.467821,375.471731,223.420193,411.394293,417.448540,283.742232,668.212704,674.529828,548.738421,262.732642,534.055278,270.164095,235.579980,668.747311,432.003382,410.755096,341.015132,243.072342,464.867976,287.035114,506.661336,359.338580,343.872469,247.546841,246.663196
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028010,0.040505,0.000000,0.000000,0.045795,0.000000,0.013365,0.000000,0.041920,0.000000,0.042262,0.025535,0.000000,0.064940,0.049133,0.034170,0.000000,0.040825,0.032045,0.000000,0.521350,0.045683,0.016190,0.030570
75%,2.021500,1.492375,2.668000,1.170500,2.032000,1.411375,2.360500,2.504750,2.003500,1.880625,3.076750,1.112000,1.533750,0.828750,2.897500,0.633325,3.209625,2.686750,0.935762,3.281750,3.038500,2.283750,1.306000,2.487750,2.704750,1.625875,6.391500,3.350125,2.806750,2.784000
max,34400.000000,60775.000000,34110.000000,24400.000000,17890.000000,87070.000000,34805.000000,19265.000000,37680.000000,37755.000000,25190.000000,66460.000000,76160.000000,55510.000000,24080.000000,49600.000000,24250.000000,21320.000000,99095.000000,54500.000000,40380.000000,29230.000000,23150.000000,45570.000000,26200.000000,44110.000000,35450.000000,34950.000000,22430.000000,21010.000000


In [34]:
import math
# Apply some known values
gm['mean'] = gm.apply(lambda row: row.mean(), axis=1)
gm['median'] = gm.apply(lambda row: row.median(), axis=1)
gm['std'] = gm.apply(lambda row: row.std(), axis=1)
gm.head()

# high expression # low deviation among tissues
gm_bench = gm[(gm['median'] >= 1)  # high expression
             & (gm['median'] - 4.0*gm['std'] > 0)
            ]

#cross_tissue_genes = sorted(gm_bench['Description'].tolist())

print("Of {} genes, the top {} in stable cross-tissue expression are:".format(len(gm), len(gm_bench)))
#for gene in cross_tissue_genes:
#    print('\t{}'.format(gene))
gm_bench

Of 56202 genes, the top 15 in stable cross-tissue expression are:


,Adipose Tissue,Adrenal Gland,Bladder,Blood,Blood Vessel,Brain,Breast,Cervix Uteri,Colon,Esophagus,Fallopian Tube,Heart,Kidney,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Salivary Gland,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,mean,median,std
Description,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MRPL20,70.695,66.140,72.74,94.785,83.84,47.690,75.685,73.540,87.92,81.295,79.47,53.440,49.190,41.430,71.72,73.765,83.250,72.90,35.5200,86.81,87.425,59.93,82.39,56.56,73.240,57.875,56.49,90.250,78.63,67.28,68.811336,72.900000,17.167726
SSU72,116.950,84.020,130.70,136.350,112.10,85.165,135.100,147.850,135.10,119.650,142.90,79.230,90.460,63.320,147.50,147.300,116.250,138.60,63.8550,156.40,134.650,116.80,151.00,113.10,151.700,100.400,111.30,188.400,142.70,139.60,120.663136,130.700000,32.070338
LAMTOR2,38.945,46.730,39.00,51.950,39.83,25.075,37.905,36.720,39.80,34.090,37.99,23.540,34.860,42.850,44.38,23.470,41.985,22.07,23.6650,33.08,38.690,35.29,56.56,33.45,46.845,37.820,26.83,38.905,32.65,34.47,35.829203,37.820000,9.269480
NDUFA10,36.365,47.125,34.45,47.730,29.08,20.490,36.180,36.130,34.55,36.270,34.23,41.190,31.050,22.060,29.92,38.515,51.375,45.87,16.8600,39.06,33.365,31.72,40.60,33.43,33.360,34.030,39.13,38.020,40.79,38.20,34.871357,36.130000,8.608415
SMIM20,31.340,33.805,36.26,28.415,29.02,21.070,30.800,27.105,32.57,29.270,30.37,21.850,27.230,15.760,26.40,35.875,32.200,43.97,19.5450,20.81,28.915,25.19,38.95,28.95,27.670,30.980,30.39,36.365,33.89,23.61,28.579764,29.020000,6.957453
MAPK9,14.900,14.280,17.94,16.700,17.77,19.440,13.205,14.840,18.61,15.620,14.33,9.805,7.936,5.535,15.22,13.125,15.660,12.62,10.5050,14.76,14.855,12.06,16.02,11.68,11.165,11.845,14.53,18.190,15.96,12.59,13.742108,14.530000,3.521462
MRPL2,31.755,33.640,33.22,47.835,30.86,15.255,33.710,31.555,31.03,30.740,34.08,26.810,29.420,30.880,26.20,38.980,34.625,32.16,15.3300,35.51,29.475,26.88,35.84,27.54,27.075,24.655,44.30,41.105,33.89,29.14,30.694066,31.030000,7.685099
POLR2J,60.865,77.315,61.73,80.575,77.56,53.210,61.495,68.050,69.07,67.150,67.46,53.940,49.140,44.010,60.03,88.770,66.285,79.25,26.3200,85.07,78.435,55.01,77.21,51.97,71.890,53.790,65.93,80.545,71.99,64.79,64.084680,66.285000,15.569742
SURF1,49.135,52.440,59.86,40.215,62.39,38.855,47.055,43.305,60.99,58.805,51.81,28.550,43.240,42.530,47.29,35.370,56.945,47.22,37.6750,57.11,57.705,44.60,35.63,46.01,54.705,54.625,53.50,66.290,48.69,41.43,47.577690,47.577434,10.933614


In [10]:
gm_high.describe()

,Adipose Tissue,Adrenal Gland,Bladder,Blood,Blood Vessel,Brain,Breast,Cervix Uteri,Colon,Esophagus,Fallopian Tube,Heart,Kidney,Liver,Lung,Muscle,Nerve,Ovary,Pancreas,Pituitary,Prostate,Salivary Gland,Skin,Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,mean,median,std
count,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,57.482084,61.009490,61.054772,70.413728,67.232188,44.460777,58.187767,65.094688,65.553277,62.208163,64.307267,48.227327,49.869198,39.082465,58.160931,55.997941,68.502634,64.845020,28.889570,66.407485,65.541609,51.124396,68.409851,51.391554,61.023248,50.520064,58.558376,71.660262,68.506822,60.646030,58.812300,59.437609,13.860251
std,58.020063,61.888289,57.782149,75.223936,76.139214,53.960162,58.470301,66.911595,69.775224,65.171923,65.520329,64.914681,53.124755,41.817030,58.780595,61.838328,69.461484,69.351918,32.633336,77.577985,70.398163,53.042895,70.917735,53.854307,64.197171,52.147076,60.890007,76.380409,68.532964,63.499728,60.652899,60.174968,14.217148
min,2.008500,2.365500,2.416000,1.808500,3.669000,1.274000,1.900000,2.212500,2.070000,2.237500,2.173000,1.548000,1.633000,1.914000,1.839000,1.990000,2.596000,2.043000,0.929600,2.236000,2.203500,1.695000,2.055000,1.529000,1.587500,2.016500,3.066000,2.360500,2.372000,2.093000,2.061370,2.055000,0.493947
25%,21.090000,20.835000,23.290000,25.270000,21.970000,14.515000,21.515000,24.875000,24.200000,22.895000,24.080000,13.830000,15.460000,13.700000,20.940000,16.510000,24.680000,22.070000,10.215000,21.880000,24.025000,18.990000,25.520000,19.140000,24.185000,16.985000,20.220000,27.570000,25.310000,23.210000,22.780450,23.466050,5.395549
50%,37.940000,44.750000,40.770000,47.835000,40.950000,23.960000,37.510000,40.660000,38.150000,36.960000,42.050000,27.310000,31.050000,25.770000,37.440000,33.560000,43.550000,40.580000,17.585000,41.480000,40.530000,33.920000,42.030000,33.430000,40.780000,34.030000,35.480000,44.550000,42.280000,39.740000,37.308667,39.590000,9.287707
75%,68.815000,69.085000,74.950000,76.940000,78.750000,53.210000,67.915000,81.790000,77.090000,75.035000,79.470000,51.380000,54.080000,42.850000,68.860000,64.510000,83.250000,78.810000,36.115000,83.150000,80.325000,59.930000,84.060000,57.650000,72.690000,57.055000,73.630000,81.855000,78.800000,70.560000,67.735667,68.520000,16.263994
max,276.050000,355.000000,290.700000,420.450000,407.200000,281.600000,277.150000,335.000000,359.700000,343.600000,340.200000,341.100000,265.700000,206.900000,292.700000,312.950000,370.550000,389.100000,222.900000,471.400000,395.650000,290.200000,381.500000,291.700000,387.650000,245.400000,380.100000,420.350000,361.800000,309.200000,299.504500,300.600000,75.012766


In [11]:

gt = gtpm[gtpm.columns[1:]]
gt.set_index('Description', inplace=True)
gtt = gt.transpose()
gtt.head()

Description,FAM138A,TP53
GTEX-1117F-0226-SM-5GZZ7,0.05045,24.98
GTEX-111CU-1826-SM-5GZYN,0.00000,35.05
GTEX-111FC-0226-SM-5N9B8,0.02945,40.62
GTEX-111VG-2326-SM-5N9BK,0.03260,29.15
GTEX-111YS-2426-SM-5GZZQ,0.00000,20.39


In [12]:
gtt.head().iplot(kind='hist', yTitle='TPM')

In [13]:
print(gt.head())
gt.iplot(kind='hist', yTitle='TPM')


             GTEX-1117F-0226-SM-5GZZ7  GTEX-111CU-1826-SM-5GZYN  \
Description                                                       
FAM138A                       0.05045                      0.00   
TP53                         24.98000                     35.05   

             GTEX-111FC-0226-SM-5N9B8  GTEX-111VG-2326-SM-5N9BK  \
Description                                                       
FAM138A                       0.02945                    0.0326   
TP53                         40.62000                   29.1500   

             GTEX-111YS-2426-SM-5GZZQ  GTEX-1122O-2026-SM-5NQ91  \
Description                                                       
FAM138A                          0.00                       0.0   
TP53                            20.39                      19.8   

             GTEX-1128S-2126-SM-5H12U  GTEX-113IC-0226-SM-5HL5C  \
Description                                                       
FAM138A                        0.0861                     0

In [14]:
gtt['TP53'].iplot(kind='hist', yTitle='TPM')